# Despliegue: Modelo de clasificación para registros de textos de mantenimiento

- Carga de los datos nuevos
- Preparación de los datos nuevos
- Carga del modelo
- Aplicación del modelo de predicción

**Preparar notebook**

In [ ]:
## Conectar el notebook a googledrive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
## Importar librerias necesarias

import nltk
import pandas as pd
import numpy as np
import re
import codecs
import pickle

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

from nltk.corpus import stopwords
stop_words_nltk = set(stopwords.words('spanish'))

from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

**Cargar los datos nuevos**

In [ ]:
#Carga del set completo de datos nuevos
df = pd.read_excel('/content/drive/MyDrive/ProyectoIntegrador/Datos/export_nuevo.xlsx')
#Habilitar la siguiente línea para correr el proyecto de manera local
#df = pd.read_excel('/content/export_nuevo.xlsx')
print(df.shape)
df.head()

/usr/local/lib/python3.8/dist-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


(239, 17)


,REPORTDATE,WONUM,DESCRIPTION,STATUS,WORKTYPE,ASSETNUM,LOCATION,REGION,LATITUDEY,LONGITUDEX,ANCESTOR,SOURCESYSID,FAILURECODE,PROBLEMCODE,RESUMEN,WORKLOGID,DETALLE
0,02/12/2022 06:03:00,13976247,Atención Evento M_EPM_EN_5128890,COMPNOREALIZADA,CINME,NaN,2088591-4,NORDESTE,6.912641,-75.085584,109-12,MAR,NaN,NaN,CIERREOT,3297751.0,Se cancela esta OT se atendió Co el evento 512...
1,02/12/2022 14:23:00,13975653,Atención Evento M_EPM_EN_5129424,EJECUTADO,CPROG,NaN,2088023-4,SUROESTE,6.108913,-75.708580,502-13,MAR,POSTE,LADEADO,CIERREOT,3297396.0,En este punto se encuentra un poste de concret...
2,02/12/2022 10:51:00,13974710,Atención Evento M_EPM_EN_5128953,EJECUTADO,CINME,NaN,32420-4,METROPOLITANA,6.208237,-75.580336,R05-10,MAR,RED.SEC,FALL.LIN,CIERREOT,3297015.0,Cámara 1072:....se diagnostica transformador #...
3,02/12/2022 08:50:00,13975654,Atención Evento M_EPM_EN_5129027,EJECUTADO,CINME,NaN,2088552-4,NORDESTE,6.495851,-74.823028,106-11,MAR,RED.PRI,FALL.LIN,CIERREOT,3297397.0,Se reparo pendiente -5128969- realizó inspec...
4,02/12/2022 12:19:00,13974497,Atención Evento M_EPM_EN_5128411,EJECUTADO,CINME,NaN,219-12,ORIENTE,6.056780,-75.182504,219-12,MAR,RED.PRI,FALL.LIN,CIERREOT,3296990.0,Se realizó acompañamiento a la ed0245 y a cuad...


In [ ]:
#Extracción de las columnas que van al modelo
datos = df[['WORKLOGID', 'DETALLE']]

**Preparar los datos nuevos**

Limpieza de texto

In [ ]:
## Definición del listado de palabras vacías de nltk
w_vacias = list(stop_words_nltk)

##carga diccionario de sinonimos
sinonimos = pd.read_csv('/content/drive/MyDrive/ProyectoIntegrador/Datos/sinonimos.csv')
#Habilitar la siguiente línea para correr el proyecto de manera local
#sinonimos = pd.read_csv('/content/sinonimos.csv')


## Definición del listado de otras palabras vacías
palabras_vacias = pd.read_csv('/content/drive/MyDrive/ProyectoIntegrador/Datos/palabras_vacias.csv')
#Habilitar la siguiente línea para correr el proyecto de manera local
#sinonimos = pd.read_csv('/content/palabras_vacias.csv')

In [ ]:
## DEFINICIÓN DE FUNCIONES PARA LIMPIEZA DE TEXTO

# Defición de función para remover del listado de tokens stopwords y tokens de longitud menor que "n" 
def remover_stopwords (tokens):
  n=2
  for i in range(len(tokens)):
    tokens[i] = [w for w in tokens[i] if w not in w_vacias and len(w)>n]
  return tokens

# Defición de función para cambiar palabras por "sinónimos"
def cambiar_sinonimos(path):
  for index in range(sinonimos.shape[0]):
    palabra = sinonimos.iloc[index, 0 ]
    sinonimo = sinonimos.iloc[index,1 ]
    path = re.sub(r'\b'+palabra+r'\b', sinonimo , path)
    ##path = path.replace(palabra, sinonimo)
  return path

# Definición de la función para la limpieza del texto en la columna "DETALLE"
def limpiar_texto (data):
  data['procesado'] = data['DETALLE'] # Crear columna para almacenar el texto procesado
  data['procesado'] = data['procesado'].apply(lambda x: re.sub('<[^>]*>', ' ', str(x))) #Remover Tags HTML
  data['procesado'] = data['procesado'].apply(lambda x: str(x).lower()) #Normalizar texto a minúscula
  data['procesado'] = data['procesado'].apply(lambda x: str(x).translate(str.maketrans('áéíóúüñ','aeiouun'))) #Cambiar tildes y ñ
  data['procesado'] = data['procesado'].apply(lambda x: re.sub('[^A-Za-z]+', ' ', str(x))) #Retirar todos los caracteres no letras
  data['procesado']=data['procesado'].apply(lambda x: cambiar_sinonimos(x)) # aplicar la función cambiar_sinonimos
  #data['procesado']=data['procesado'].apply(lambda x: corregir_texto(x, corrector=corrector))
  data['tokens'] = data['procesado'].apply(lambda x: nltk.word_tokenize(str(x))) #Crear columna con los tokens
  data['tokens_proc'] = data['tokens'] # Crear columna para almacenar los tokens procesados
  data['tokens_proc'] = remover_stopwords(data['tokens_proc']) # Remover stopwords y tokens "cortos" del listado de tokens
  return data

# Defición de función que elimina listado de tokens correspondientes a "palabras vacias" 
def remover_palabra (tokens):
  for i in range(len(tokens)):
    tokens[i] = [w for w in tokens[i] if w not in palabras_vacias]
  return tokens



In [ ]:
# Limpiar el texto en la columna "DETALLE"
limpiar_texto(datos)

<ipython-input-8-3e16f1716ce3>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['procesado'] = data['DETALLE'] # Crear columna para almacenar el texto procesado
<ipython-input-8-3e16f1716ce3>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['procesado'] = data['procesado'].apply(lambda x: re.sub('<[^>]*>', ' ', str(x))) #Remover Tags HTML
<ipython-input-8-3e16f1716ce3>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_i

,WORKLOGID,DETALLE,procesado,tokens,tokens_proc
0,3297751.0,Se cancela esta OT se atendió Co el evento 512...,se cancela esta ot se atendio co el evento,"[se, cancela, esta, ot, se, atendio, co, el, e...","[cancela, atendio, evento]"
1,3297396.0,En este punto se encuentra un poste de concret...,en este punto se encuentra un poste de concret...,"[en, este, punto, se, encuentra, un, poste, de...","[punto, encuentra, poste, concreto, ladeado, e..."
2,3297015.0,Cámara 1072:....se diagnostica transformador #...,camara se diagnostica transformador con vs nom...,"[camara, se, diagnostica, transformador, con, ...","[camara, diagnostica, transformador, nominal, ..."
3,3297397.0,Se reparo pendiente -5128969- realizó inspec...,se repara pendiente realizo inspeccion en la r...,"[se, repara, pendiente, realizo, inspeccion, e...","[repara, pendiente, realizo, inspeccion, ruta,..."
4,3296990.0,Se realizó acompañamiento a la ed0245 y a cuad...,se realizo acompanamiento a la ed y a cuadrill...,"[se, realizo, acompanamiento, a, la, ed, y, a,...","[realizo, acompanamiento, cuadrilla, contratis..."
...,...,...,...,...,...
234,3297411.0,Ed 0130 se verifica información de la ot y en ...,ed se verifica informacion de la ot y en la ve...,"[ed, se, verifica, informacion, de, la, ot, y,...","[verifica, informacion, vereda, encuentran, re..."
235,3296747.0,Se reviso y se encontró poste secundario metál...,se reviso y se encontro poste secundario metal...,"[se, reviso, y, se, encontro, poste, secundari...","[reviso, encontro, poste, secundario, metalico..."
236,3296170.0,Se cambió fusible 1x10T en el FR 45826 se norm...,se cambia fusible x t en el fr se normaliza el...,"[se, cambia, fusible, x, t, en, el, fr, se, no...","[cambia, fusible, normaliza, servicio, mal, so..."
237,3296661.0,Se realizó inspección para la poda de un árbo...,se realizo inspeccion para la poda de un arbol...,"[se, realizo, inspeccion, para, la, poda, de, ...","[realizo, inspeccion, poda, arbol, riesgo, cae..."


In [ ]:
datos['tokens_proc'] = remover_palabra(datos['tokens_proc'])

<ipython-input-8-3e16f1716ce3>:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tokens[i] = [w for w in tokens[i] if w not in palabras_vacias]
<ipython-input-10-955b6c96719c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datos['tokens_proc'] = remover_palabra(datos['tokens_proc'])


In [ ]:
datos.head()

,WORKLOGID,DETALLE,procesado,tokens,tokens_proc
0,3297751.0,Se cancela esta OT se atendió Co el evento 512...,se cancela esta ot se atendio co el evento,"[se, cancela, esta, ot, se, atendio, co, el, e...","[cancela, atendio, evento]"
1,3297396.0,En este punto se encuentra un poste de concret...,en este punto se encuentra un poste de concret...,"[en, este, punto, se, encuentra, un, poste, de...","[punto, encuentra, poste, concreto, ladeado, e..."
2,3297015.0,Cámara 1072:....se diagnostica transformador #...,camara se diagnostica transformador con vs nom...,"[camara, se, diagnostica, transformador, con, ...","[camara, diagnostica, transformador, nominal, ..."
3,3297397.0,Se reparo pendiente -5128969- realizó inspec...,se repara pendiente realizo inspeccion en la r...,"[se, repara, pendiente, realizo, inspeccion, e...","[repara, pendiente, realizo, inspeccion, ruta,..."
4,3296990.0,Se realizó acompañamiento a la ed0245 y a cuad...,se realizo acompanamiento a la ed y a cuadrill...,"[se, realizo, acompanamiento, a, la, ed, y, a,...","[realizo, acompanamiento, cuadrilla, contratis..."


In [ ]:
def unir_tokens (text):
  doc = ' '.join(text)
  return doc

In [ ]:
##aplicar la función de unir tokens##
datos['tokensunidos']=datos['tokens_proc'].apply(lambda x: unir_tokens(x))
datos.head()

<ipython-input-13-2fb83948b118>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datos['tokensunidos']=datos['tokens_proc'].apply(lambda x: unir_tokens(x))


,WORKLOGID,DETALLE,procesado,tokens,tokens_proc,tokensunidos
0,3297751.0,Se cancela esta OT se atendió Co el evento 512...,se cancela esta ot se atendio co el evento,"[se, cancela, esta, ot, se, atendio, co, el, e...","[cancela, atendio, evento]",cancela atendio evento
1,3297396.0,En este punto se encuentra un poste de concret...,en este punto se encuentra un poste de concret...,"[en, este, punto, se, encuentra, un, poste, de...","[punto, encuentra, poste, concreto, ladeado, e...",punto encuentra poste concreto ladeado edatel ...
2,3297015.0,Cámara 1072:....se diagnostica transformador #...,camara se diagnostica transformador con vs nom...,"[camara, se, diagnostica, transformador, con, ...","[camara, diagnostica, transformador, nominal, ...",camara diagnostica transformador nominal salid...
3,3297397.0,Se reparo pendiente -5128969- realizó inspec...,se repara pendiente realizo inspeccion en la r...,"[se, repara, pendiente, realizo, inspeccion, e...","[repara, pendiente, realizo, inspeccion, ruta,...",repara pendiente realizo inspeccion ruta repar...
4,3296990.0,Se realizó acompañamiento a la ed0245 y a cuad...,se realizo acompanamiento a la ed y a cuadrill...,"[se, realizo, acompanamiento, a, la, ed, y, a,...","[realizo, acompanamiento, cuadrilla, contratis...",realizo acompanamiento cuadrilla contratista c...


In [ ]:
X_new = datos.tokensunidos
X_new.shape

(239,)

**Cargar el modelo**

In [ ]:
import pickle
filename = 'modelo.pkl'
knn, nb, rl, variables,countvec,tfidf = pickle.load(open(filename, 'rb'))

In [ ]:
Xcv = countvec.transform(X_new)
Xtfidf = tfidf.fit_transform(Xcv)

**Usar el modelo para la clasificación automática de los registros nuevos**

In [ ]:
y_pred = nb.predict(Xtfidf)

In [ ]:
datos['pred_pendiente'] = y_pred
datos.head()

<ipython-input-18-806f58b50271>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datos['pred_pendiente'] = y_pred


,WORKLOGID,DETALLE,procesado,tokens,tokens_proc,tokensunidos,pred_pendiente
0,3297751.0,Se cancela esta OT se atendió Co el evento 512...,se cancela esta ot se atendio co el evento,"[se, cancela, esta, ot, se, atendio, co, el, e...","[cancela, atendio, evento]",cancela atendio evento,0
1,3297396.0,En este punto se encuentra un poste de concret...,en este punto se encuentra un poste de concret...,"[en, este, punto, se, encuentra, un, poste, de...","[punto, encuentra, poste, concreto, ladeado, e...",punto encuentra poste concreto ladeado edatel ...,0
2,3297015.0,Cámara 1072:....se diagnostica transformador #...,camara se diagnostica transformador con vs nom...,"[camara, se, diagnostica, transformador, con, ...","[camara, diagnostica, transformador, nominal, ...",camara diagnostica transformador nominal salid...,1
3,3297397.0,Se reparo pendiente -5128969- realizó inspec...,se repara pendiente realizo inspeccion en la r...,"[se, repara, pendiente, realizo, inspeccion, e...","[repara, pendiente, realizo, inspeccion, ruta,...",repara pendiente realizo inspeccion ruta repar...,1
4,3296990.0,Se realizó acompañamiento a la ed0245 y a cuad...,se realizo acompanamiento a la ed y a cuadrill...,"[se, realizo, acompanamiento, a, la, ed, y, a,...","[realizo, acompanamiento, cuadrilla, contratis...",realizo acompanamiento cuadrilla contratista c...,1


In [ ]:
pendientes = datos['WORKLOGID'][datos['pred_pendiente']==1]
pendientes.shape

(97,)

In [ ]:
df_pendientes = df[df['WORKLOGID'].isin(pendientes)]
df_pendientes

,REPORTDATE,WONUM,DESCRIPTION,STATUS,WORKTYPE,ASSETNUM,LOCATION,REGION,LATITUDEY,LONGITUDEX,ANCESTOR,SOURCESYSID,FAILURECODE,PROBLEMCODE,RESUMEN,WORKLOGID,DETALLE
2,02/12/2022 10:51:00,13974710,Atención Evento M_EPM_EN_5128953,EJECUTADO,CINME,NaN,32420-4,METROPOLITANA,6.208237,-75.580336,R05-10,MAR,RED.SEC,FALL.LIN,CIERREOT,3297015.0,Cámara 1072:....se diagnostica transformador #...
3,02/12/2022 08:50:00,13975654,Atención Evento M_EPM_EN_5129027,EJECUTADO,CINME,NaN,2088552-4,NORDESTE,6.495851,-74.823028,106-11,MAR,RED.PRI,FALL.LIN,CIERREOT,3297397.0,Se reparo pendiente -5128969- realizó inspec...
4,02/12/2022 12:19:00,13974497,Atención Evento M_EPM_EN_5128411,EJECUTADO,CINME,NaN,219-12,ORIENTE,6.056780,-75.182504,219-12,MAR,RED.PRI,FALL.LIN,CIERREOT,3296990.0,Se realizó acompañamiento a la ed0245 y a cuad...
6,02/12/2022 08:05:00,13973487,Atención Evento M_EPM_EN_5128982,EJECUTADO,CINME,NaN,503-13,SUROESTE,5.928851,-75.670007,503-13,MAR,RED.SEC,FALL.LIN,CIERREOT,3296493.0,Se acciono protección abiertas por descarga at...
8,02/12/2022 08:40:00,13971878,Atención Evento M_EPM_EN_5129014,EJECUTADO,CPROG,3158117.0,2088551-5,NORTE,7.176526,-75.764044,305-12,MAR,POSTE,LADEADO,CIERREOT,3295835.0,Se requiere cambiar y reubicar poste sec.de 8m...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
229,02/12/2022 09:16:00,13974463,Atención Evento M_EPM_EN_5129069,EJECUTADO,CINME,NaN,54920,ORIENTE,6.296743,-75.065064,R50-02,MAR,CAJA,CBL EXP,CIERREOT,3296974.0,Serealiso poda Serca del trafo 54920 quedó li...
231,02/12/2022 08:44:00,13973546,Atención Evento M_EPM_EN_5129021,EJECUTADO,CINME,129748.0,130010,NORDESTE,6.561822,-74.968733,104-13,MAR,RED.PRI,FALL.LIN,CIERREOT,3296545.0,Se cambia fusible en trafo 129748 se gasta de...
232,02/12/2022 14:30:00,13976021,Atención Evento M_EPM_EN_5129433,EJECUTADO,CPROG,NaN,908132,BAJO CAUCA,7.403468,-74.882845,313-12,MAR,RED.SEC,FALL.LIN,CIERREOT,3297654.0,Se llega al sitio y se encontró triplex epor c...
235,02/12/2022 09:01:00,13973857,Atención Evento M_EPM_EN_5129050,EJECUTADO,CPROG,695029.0,695029,OCCIDENTE,6.679900,-75.973355,607-12,MAR,POSTE,PELIGRO,CIERREOT,3296747.0,Se reviso y se encontró poste secundario metál...


**Exportación de los datos clasificados como pendientes para ser usados en aplicaciones de visualización**

In [ ]:
df_pendientes.to_csv('/content/drive/MyDrive/ProyectoIntegrador/Datos/prediccion_pendientes.csv')
#Habilitar la siguiente línea para correr el proyecto de manera local
#df_pendientes.to_csv('/content/prediccion_pendientes.csv')